In [1]:
import os
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset
from sklearn.metrics import accuracy_score, f1_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_directory = 'G:/Data Projects/LML_text_classification/Data/uci-reuters-21578-text-categorization-collection/uci-reuters-21578-text-categorization-collection/original/selected'

In [3]:
sgm_files = [f for f in os.listdir(data_directory) if f.endswith('.sgm')]

text_data = []
category_data = []

for sgm_file in sgm_files:
    file_path = os.path.join(data_directory, sgm_file)
    
    with open(file_path, 'r', encoding='latin-1') as file:
        content = file.read()

    soup = BeautifulSoup(content, 'html.parser')
    articles = soup.find_all('reuters')
    
    for article in articles:
        text = article.find('text').get_text()
        text_data.append(text)
        
        topics = article.find_all('topics')
        categories = [topic.get_text() for topic in topics]
        category_data.append(categories)
      

In [4]:
print(category_data)

[['cocoa'], [''], [''], [''], ['grainwheatcornbarleyoatsorghum'], ['veg-oillinseedlin-oilsoy-oilsun-oilsoybeanoilseedcornsunseedgrainsorghumwheat'], [''], [''], ['earn'], ['acq'], ['earn'], ['earnacq'], ['earn'], ['earn'], [''], [''], [''], ['earn'], ['wheatgrain'], [''], [''], ['copper'], ['earn'], ['earn'], [''], [''], ['earn'], [''], ['housing'], ['money-supply'], [''], [''], [''], [''], [''], ['earn'], ['earn'], ['earn'], [''], ['earn'], ['earn'], ['coffee'], [''], ['acqship'], ['acq'], ['sugar'], ['trade'], ['reserves'], ['ship'], ['earn'], [''], [''], ['earn'], [''], [''], ['earn'], ['graincorn'], ['money-supply'], ['ship'], [''], [''], [''], [''], ['earn'], ['earn'], ['earn'], [''], ['acq'], ['veg-oilsoybeanoilseedmeal-feedsoy-meal'], [''], ['earn'], [''], [''], ['earn'], ['coffee'], ['money-supply'], ['money-supply'], ['money-supply'], [''], ['money-supply'], ['money-supply'], ['earn'], ['earn'], [''], ['earn'], ['earn'], ['earn'], [''], ['earn'], [''], [''], ['money-supply'], 

In [5]:
category_data = [category[0] if category else 'uncategorized' for category in category_data]
print(category_data)

['cocoa', '', '', '', 'grainwheatcornbarleyoatsorghum', 'veg-oillinseedlin-oilsoy-oilsun-oilsoybeanoilseedcornsunseedgrainsorghumwheat', '', '', 'earn', 'acq', 'earn', 'earnacq', 'earn', 'earn', '', '', '', 'earn', 'wheatgrain', '', '', 'copper', 'earn', 'earn', '', '', 'earn', '', 'housing', 'money-supply', '', '', '', '', '', 'earn', 'earn', 'earn', '', 'earn', 'earn', 'coffee', '', 'acqship', 'acq', 'sugar', 'trade', 'reserves', 'ship', 'earn', '', '', 'earn', '', '', 'earn', 'graincorn', 'money-supply', 'ship', '', '', '', '', 'earn', 'earn', 'earn', '', 'acq', 'veg-oilsoybeanoilseedmeal-feedsoy-meal', '', 'earn', '', '', 'earn', 'coffee', 'money-supply', 'money-supply', 'money-supply', '', 'money-supply', 'money-supply', 'earn', 'earn', '', 'earn', 'earn', 'earn', '', 'earn', '', '', 'money-supply', 'earn', 'money-supply', 'money-supply', 'acq', 'grainwheatcornoatryesorghumsoybeanoilseed', 'earn', 'money-supply', 'money-supply', 'money-supply', 'money-supply', 'money-supply', 'cot

In [6]:
def clean_categories(category_str):
    categories = re.split(r'(?<=\w)(?=[A-Z])|(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])', category_str)
    return [cat.strip() for cat in categories if cat.strip()]
category_data = [clean_categories(entry) for entry in category_data]

category_data = [categories if categories else ['uncategorized'] for categories in category_data]

print(category_data)


[['cocoa'], ['uncategorized'], ['uncategorized'], ['uncategorized'], ['grainwheatcornbarleyoatsorghum'], ['veg-oillinseedlin-oilsoy-oilsun-oilsoybeanoilseedcornsunseedgrainsorghumwheat'], ['uncategorized'], ['uncategorized'], ['earn'], ['acq'], ['earn'], ['earnacq'], ['earn'], ['earn'], ['uncategorized'], ['uncategorized'], ['uncategorized'], ['earn'], ['wheatgrain'], ['uncategorized'], ['uncategorized'], ['copper'], ['earn'], ['earn'], ['uncategorized'], ['uncategorized'], ['earn'], ['uncategorized'], ['housing'], ['money-supply'], ['uncategorized'], ['uncategorized'], ['uncategorized'], ['uncategorized'], ['uncategorized'], ['earn'], ['earn'], ['earn'], ['uncategorized'], ['earn'], ['earn'], ['coffee'], ['uncategorized'], ['acqship'], ['acq'], ['sugar'], ['trade'], ['reserves'], ['ship'], ['earn'], ['uncategorized'], ['uncategorized'], ['earn'], ['uncategorized'], ['uncategorized'], ['earn'], ['graincorn'], ['money-supply'], ['ship'], ['uncategorized'], ['uncategorized'], ['uncategor

In [7]:
print(text_data[0])


BAHIA COCOA REVIEW
    SALVADOR, Feb 26 - Showers continued throughout the week in
the Bahia cocoa zone, alleviating the drought since early
January and improving prospects for the coming temporao,
although normal humidity levels have not been restored,
Comissaria Smith said in its weekly review.
    The dry period means the temporao will be late this year.
    Arrivals for the week ended February 22 were 155,221 bags
of 60 kilos making a cumulative total for the season of 5.93
mln against 5.81 at the same stage last year. Again it seems
that cocoa delivered earlier on consignment was included in the
arrivals figures.
    Comissaria Smith said there is still some doubt as to how
much old crop cocoa is still available as harvesting has
practically come to an end. With total Bahia crop estimates
around 6.4 mln bags and sales standing at almost 6.2 mln there
are a few hundred thousand bags still in the hands of farmers,
middlemen, exporters and processors.
    There are doubts as to how

In [8]:
#checking for empty text strings
for index, text in enumerate(text_data):
    if not text.strip():
        print(f"Empty string found at index {index}: {text}")


In [9]:
def preprocess_text(text):
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    words = text.split()
    words = [word.lower() for word in words]
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    processed_text = ' '.join(words)
    
    return processed_text

preprocessed_data = [preprocess_text(text) for text in text_data]

In [10]:
print(preprocessed_data[0])

bahia cocoa review salvador feb 26 showers continued throughout week bahia cocoa zone alleviating drought since early january improving prospects coming temporao although normal humidity levels restored comissaria smith said weekly review dry period means temporao late year arrivals week ended february 22 155 221 bags 60 kilos making cumulative total season 5 93 mln 5 81 stage last year seems cocoa delivered earlier consignment included arrivals figures comissaria smith said still doubt much old crop cocoa still available harvesting practically come end total bahia crop estimates around 6 4 mln bags sales standing almost 6 2 mln hundred thousand bags still hands farmers middlemen exporters processors doubts much cocoa would fit export shippers experiencing dificulties obtaining bahia superior certificates view lower quality recent weeks farmers sold good part cocoa held consignment comissaria smith said spot bean prices rose 340 350 cruzados per arroba 15 kilos bean shippers reluctant 

In [11]:
category_data_flat = [category[0] for category in category_data]
label_mapping = {}
unique_labels = set(category_data_flat) 

for i, label in enumerate(unique_labels):
    label_mapping[label] = i

numerical_labels = [label_mapping[label[0]] for label in category_data]
print("Number of unique labels:", len(unique_labels))

Number of unique labels: 232


In [12]:
X_train, X_temp, y_train, y_temp = train_test_split(preprocessed_data, numerical_labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [13]:
tfidf_vectorizer = TfidfVectorizer(max_features=512, stop_words='english')
X_tfidf = tfidf_vectorizer.fit_transform(preprocessed_data)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [15]:
X_train_tfidf = torch.tensor(X_train_tfidf.toarray(), dtype=torch.long).view(-1, 512)
X_val_tfidf = torch.tensor(X_val_tfidf.toarray(), dtype=torch.long).view(-1, 512)
y_train = torch.tensor(y_train, dtype=torch.long)
y_val = torch.tensor(y_val).to(torch.long)

train_dataset = TensorDataset(X_train_tfidf, y_train)
val_dataset = TensorDataset(X_val_tfidf, y_val)

batch_size = 32
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size)


In [16]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(unique_labels))
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3 
best_validation_loss = float('inf')
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        inputs, labels = batch
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    model.eval()
    validation_losses = []
    predictions = []
    true_labels = []
    
    with torch.no_grad():
        for batch in val_dataloader:
            inputs, labels = batch
            outputs = model(inputs, labels=labels)
            loss = outputs.loss
            validation_losses.append(loss.item())
            
            logits = outputs.logits
            predicted_labels = torch.argmax(logits, dim=1)
            predictions.extend(predicted_labels.tolist())
            true_labels.extend(labels.tolist())
    validation_accuracy = accuracy_score(true_labels, predictions)
    validation_f1 = f1_score(true_labels, predictions, average='weighted')
    
    print(f"Epoch {epoch + 1}: Validation Loss: {sum(validation_losses) / len(validation_losses):.4f}")
    print(f"Validation Accuracy: {validation_accuracy:.4f}")
    print(f"Validation F1 Score: {validation_f1:.4f}")
    if sum(validation_losses) < best_validation_loss:
        best_validation_loss = sum(validation_losses)
        # Save the best model checkpoint
        model.save_pretrained(data_directory)

best_model = BertForSequenceClassification.from_pretrained(data_directory)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch 1: Validation Loss: 2.4140
Validation Accuracy: 0.4627
Validation F1 Score: 0.2927
Epoch 2: Validation Loss: 2.2912
Validation Accuracy: 0.4627
Validation F1 Score: 0.2927
Epoch 3: Validation Loss: 2.2629
Validation Accuracy: 0.4627
Validation F1 Score: 0.2927
